In [2]:
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                

In [29]:

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0           conda-forge
    geopy:         1.22.0-pyh9f0ad1d_0 conda-forge


geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.22.0         | 63 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 1

In [8]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [9]:
soup = BeautifulSoup(data, 'html.parser')

In [10]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [11]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [12]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [13]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [37]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))

In [38]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

# Check to make sure the coordinates are added as required by the 2nd question

In [39]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)

In [40]:
toronto_df_grouped.shape

(180, 3)

In [41]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned\n,NaN,NaN
1,M1B\n,Scarborough\n,"Malvern, Rouge",NaN,NaN
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",NaN,NaN
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",NaN,NaN
4,M1G\n,Scarborough\n,Woburn,NaN,NaN


In [32]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [42]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [47]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto\n', 'Central Toronto\n', 'Downtown Toronto\n', 'West Toronto\n']

In [48]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",NaN,NaN
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",NaN,NaN
3,M4M\n,East Toronto\n,Studio District,NaN,NaN
4,M4N\n,Central Toronto\n,Lawrence Park,NaN,NaN


In [50]:
CLIENT_ID = 'ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ' # your Foursquare ID
CLIENT_SECRET = 'OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ
CLIENT_SECRET:OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H


In [51]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ&client_secret=OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [52]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1833, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Textile Museum of Canada,43.654396,-79.386500,Art Museum
2,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Rolltation,43.654918,-79.387424,Japanese Restaurant
3,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Cafe Plenty,43.654571,-79.389450,Café
4,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant


In [53]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E\n,East Toronto\n,The Beaches,0,0,47,47,47,47
M4K\n,East Toronto\n,"The Danforth West, Riverdale",0,0,47,47,47,47
M4L\n,East Toronto\n,"India Bazaar, The Beaches West",0,0,47,47,47,47
M4M\n,East Toronto\n,Studio District,0,0,47,47,47,47
M4N\n,Central Toronto\n,Lawrence Park,0,0,47,47,47,47
M4P\n,Central Toronto\n,Davisville North,0,0,47,47,47,47
M4R\n,Central Toronto\n,"North Toronto West, Lawrence Park",0,0,47,47,47,47
M4S\n,Central Toronto\n,Davisville,0,0,47,47,47,47
M4T\n,Central Toronto\n,"Moore Park, Summerhill East",0,0,47,47,47,47


In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1833, 38)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Opera House,Park,Plaza,Pub,Ramen Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E\n,East Toronto\n,The Beaches,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [55]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 38)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Opera House,Park,Plaza,Pub,Ramen Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E\n,East Toronto\n,The Beaches,0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277
3,M4M\n,East Toronto\n,Studio District,0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277
4,M4N\n,Central Toronto\n,Lawrence Park,0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277
5,M4P\n,Central Toronto\n,Davisville North,0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277
6,M4R\n,Central Toronto\n,"North Toronto West, Lawrence Park",0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277
7,M4S\n,Central Toronto\n,Davisville,0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277
8,M4T\n,Central Toronto\n,"Moore Park, Summerhill East",0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277
9,M4V\n,Central Toronto\n,"Summerhill West, Rathnelly, South Hill, Forest...",0.085106,0.021277,0.021277,0.021277,0.021277,0.021277,0.06383,...,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.021277,0.042553,0.021277,0.021277


In [57]:
import numpy as np

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted


(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
3,M4M\n,East Toronto\n,Studio District,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
4,M4N\n,Central Toronto\n,Lawrence Park,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
5,M4P\n,Central Toronto\n,Davisville North,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
6,M4R\n,Central Toronto\n,"North Toronto West, Lawrence Park",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
7,M4S\n,Central Toronto\n,Davisville,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
8,M4T\n,Central Toronto\n,"Moore Park, Summerhill East",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
9,M4V\n,Central Toronto\n,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


# Clustering

In [60]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [61]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
3,M4M\n,East Toronto\n,Studio District,NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
4,M4N\n,Central Toronto\n,Lawrence Park,NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


In [62]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
21,M5L\n,Downtown Toronto\n,"Commerce Court, Victoria Hotel",NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
22,M5N\n,Central Toronto\n,Roselawn,NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
23,M5P\n,Central Toronto\n,"Forest Hill North & West, Forest Hill Road Park",NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
24,M5R\n,Central Toronto\n,"The Annex, North Midtown, Yorkville",NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
25,M5S\n,Downtown Toronto\n,"University of Toronto, Harbord",NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
26,M5T\n,Downtown Toronto\n,"Kensington Market, Chinatown, Grange Park",NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
27,M5V\n,Downtown Toronto\n,"CN Tower, King and Spadina, Railway Lands, Har...",NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
20,M5K\n,Downtown Toronto\n,"Toronto Dominion Centre, Design Exchange",NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
28,M5W\n,Downtown Toronto\n,Stn A PO Boxes,NaN,NaN,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


# CHECKING THE CLUSTERS

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
21,Downtown Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
22,Central Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
23,Central Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
24,Central Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
25,Downtown Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
26,Downtown Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
27,Downtown Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
20,Downtown Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
28,Downtown Toronto\n,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [70]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [71]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Conclusion


**Most of the neighborhoods fall into Cluster 1 which are the areas with cafe, restaurants, supermarkets etc